### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_complete_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])



## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#total number of schools 
total_unique_schools = school_complete_df["school_name"].nunique()

#Total Students column needs header in summary table
student_count = school_complete_df["student_name"].count()

In [3]:
#isolate relevant columns
school_budget_df= school_complete_df[["school_name","budget"]]

#group by min of school and budget to remove duplicate values 

budget_columns = school_budget_df.groupby(["school_name","budget"], as_index=False).min()

#sum budget column for total budget
budget_total = budget_columns["budget"].sum()
comma_budget_total = "${:,.2f}".format(budget_total)
comma_budget_total

'$24,649,428.00'

In [4]:
#calculate the average scores for math and reading
math_average = round(school_complete_df["math_score"].mean(),2)
reading_average = round(school_complete_df["reading_score"].mean(),2)

In [5]:
#find the count of students that passed math
pass_math_df= school_complete_df.loc[school_complete_df
                                        ["math_score"]>=70]

math_passed_perc = round(pass_math_df["student_name"].count()/student_count*100,2)

In [6]:
#find the count of students that passed reading
pass_reading_df= school_complete_df.loc[school_complete_df
                                        ["reading_score"]>=70]

read_passed_perc = round(pass_reading_df["student_name"].count()/student_count*100,2)

In [7]:
#use and function to find percentage of students that passed both tests
pass_both_df = school_complete_df.loc[(school_complete_df["reading_score"]>=70) & (school_complete_df["math_score"]>=70)]

both_passed_perc = round(pass_both_df["student_name"].count()/student_count*100,2)

In [8]:
#pass key titles and values to dictionary and convert to df
district_summary_dict = {"Total Schools": [total_unique_schools],"Total Students": [student_count],
                      "Total Budget": [comma_budget_total],"Average Math Score": [math_average],
                       "Average Reading Score":[reading_average],"% Passing Math":
                       [math_passed_perc],"% Passing Reading": [read_passed_perc],
                       "% Passing Overall":[both_passed_perc]}
#create data frame
district_summary_df=pd.DataFrame(district_summary_dict)
               
district_summary_df.style.format({"Total Students": "{:,}","Total School Budget":"${:,.2f}","Per Student Budget": "${:,.2f}", 
                                "Average Math Score": "{:.2f}","Average Reading Score":"{:.2f}",
                                "% Passing Math":"%{:,.2f}","% Passing Reading":"%{:,.2f}",
                                "% Passing Overall":"%{:,.2f}"})

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,"39,170","$24,649,428.00",78.99,81.88,%74.98,%85.81,%65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [9]:
#set the school name as index --HOW DO I GET RID OF THE TITLE
school_type=school_complete_df.set_index(["school_name"])["type"]
school_type=school_type.groupby(["school_name"]).min()

In [10]:
#group count of students by school 
total_students = school_complete_df.groupby(["school_name"])["student_name"].count()

In [11]:
#groupby school budget totals
school_budget = school_complete_df.groupby(["school_name"])["budget"].min()

#for budget per student divide budget total column by total students column
perstudent_budget = school_budget/total_students



# Use mean function to find average math score 
school_math_avg = round(school_complete_df.groupby(["school_name"])["math_score"].mean(),2)

# Use mean function to find average reading score 
school_reading_avg = school_complete_df.groupby(["school_name"])["reading_score"].mean()

#Find % passing math grouped by school name using the count function of students


pass_math_df= school_complete_df.loc[school_complete_df["math_score"]>=70]
perc_pass_math = round(pass_math_df.groupby("school_name").count()["student_name"]/total_students*100,2)

#Find % passing reading grouped by school name using the count function of students
pass_reading_df= school_complete_df.loc[school_complete_df["reading_score"]>=70]
perc_pass_reading = round(pass_reading_df.groupby(["school_name"]).count()["student_name"]/total_students*100,2)

#find % passing both using and statement grouped by shcool name using the count function of student name 
pass_both_df = school_complete_df.loc[(school_complete_df["reading_score"]>=70) & (school_complete_df["math_score"]>=70)]
perc_pass_both = round(pass_both_df.groupby("school_name").count()["student_name"]/total_students*100,2)                       

In [12]:
#create a dictionary with the values above
school_summary_dict = {"Type":school_type,"Total Students":total_students,
                       "Total School Budget":school_budget, 
                       "Per Student Budget": perstudent_budget,"Average Math Score": 
              
                       school_math_avg,"Average Reading Score":school_reading_avg,
                       "% Passing Math":perc_pass_math, "% Passing Reading": perc_pass_reading,
                      "% Passing Overall": perc_pass_both}
#use dictionary to create data frame
school_summary_df = pd.DataFrame(school_summary_dict)

#remove index name
school_summary_df.rename_axis(None, inplace=True)

#format data
school_summary_df.style.format({"Total Students": "{:,}","Total School Budget":"${:,.2f}","Per Student Budget": "${:,.2f}", 
                                "Average Math Score": "{:.2f}","Average Reading Score":"{:.2f}",
                                "% Passing Math":"%{:,.2f}","% Passing Reading":"%{:,.2f}",
                                "% Passing Overall":"%{:,.2f}"})

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,%66.68,%81.93,%54.64
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,%94.13,%97.04,%91.33
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,%65.99,%80.74,%53.20
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,%68.31,%79.30,%54.29
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,%93.39,%97.14,%90.60
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,%66.75,%80.86,%53.53
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,%92.51,%96.25,%89.23
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,%65.68,%81.32,%53.51
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,%66.06,%81.22,%53.54
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,%94.59,%95.95,%90.54


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [13]:
#rename and sort data frame by descending values in % Passing Overall
top_overall_df = school_summary_df.sort_values(["% Passing Overall"], ascending=False)

#remove index name
top_overall_df.rename_axis(None, inplace=True)

#display first 5 rows and all columns
top_overall_df.iloc[0:5,:].style.format({"Total Students": "{:,}","Total School Budget":"${:,.2f}",
                                         "Per Student Budget": "${:,.2f}", 
                                "Average Math Score": "{:.2f}","Average Reading Score":"{:.2f}",
                                "% Passing Math":"%{:,.2f}","% Passing Reading":"%{:,.2f}",
                                "% Passing Overall":"%{:,.2f}"})
           

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,%94.13,%97.04,%91.33
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,%93.27,%97.31,%90.95
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,%93.39,%97.14,%90.60
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,%93.87,%96.54,%90.58
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,%94.59,%95.95,%90.54


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [14]:
#rename and sort data frame by ascending values in % Passing Overall
bot_overall_df = school_summary_df.sort_values(["% Passing Overall"], ascending=True)

#remove index name
bot_overall_df.rename_axis(None, inplace=True)

#display first 5 rows and all columns
bot_overall_df.iloc[0:5,:].style.format({"Total Students": "{:,}","Total School Budget":"${:,.2f}","Per Student Budget": "${:,.2f}", 
                                "Average Math Score": "{:.2f}","Average Reading Score":"{:.2f}",
                                "% Passing Math":"%{:,.2f}","% Passing Reading":"%{:,.2f}",
                                "% Passing Overall":"%{:,.2f}"})


,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,%66.37,%80.22,%52.99
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,%65.99,%80.74,%53.20
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,%65.68,%81.32,%53.51
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,%66.75,%80.86,%53.53
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,%66.06,%81.22,%53.54


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [15]:

#filter by grade 9
grade_9 = school_complete_df.loc[school_complete_df["grade"]=="9th"]

#group by the school name to get average math score
grade_9_group = round(grade_9.groupby(["school_name"]).mean(),2)
math_9 = grade_9_group["math_score"]

#filter by grade 10
grade_10 = school_complete_df.loc[school_complete_df["grade"]=="10th"]

#group by the school name to get average math score
grade_10_group = round(grade_10.groupby(["school_name"]).mean(),2)
math_10 = grade_10_group["math_score"]

#filter by grade 11
grade_11 = school_complete_df.loc[school_complete_df["grade"]=="11th"]

#group by the school name to get average math score
grade_11_group = round(grade_11.groupby(["school_name"]).mean(),2)
math_11 = grade_11_group["math_score"]

#filter by grade 12
grade_12 = school_complete_df.loc[school_complete_df["grade"]=="12th"]

#group by the school name to get average math score
grade_12_group = round(grade_12.groupby(["school_name"]).mean(),2)
math_12 = grade_12_group["math_score"]

In [16]:
#Pass values in above cell into a dictionary
mscores_by_grade_dict = {"9th":math_9,"10th":math_10,"11th":math_11,"12th":math_12}

#use dictionary above to create data frame
mscores_by_grade_df = pd.DataFrame(mscores_by_grade_dict)

#remove index name
mscores_by_grade_df.rename_axis(None, inplace=True)

#format scores into floats with 2 decimal places
mscores_by_grade_df.style.format({"9th":"{:,.2f}","10th":"{:,.2f}","11th":"{:,.2f}","12th":"{:,.2f}"})

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [17]:
#fid the average reading score by grade
#filter by grade 9
grade_9 = school_complete_df.loc[school_complete_df["grade"]=="9th"]

#group by the school name to get average math score
grade_9_group = round(grade_9.groupby(["school_name"]).mean(),2)
reading_9 = grade_9_group["reading_score"]

#filter by grade 10
grade_10 = school_complete_df.loc[school_complete_df["grade"]=="10th"]

#group by the school name to get average math score
grade_10_group = round(grade_10.groupby(["school_name"]).mean(),2)
reading_10 = grade_10_group["reading_score"]

#filter by grade 11
grade_11 = school_complete_df.loc[school_complete_df["grade"]=="11th"]

#group by the school name to get average math score
grade_11_group = round(grade_11.groupby(["school_name"]).mean(),2)
reading_11 = grade_11_group["reading_score"]

#filter by grade 12
grade_12 = school_complete_df.loc[school_complete_df["grade"]=="12th"]

#group by the school name to get average math score
grade_12_group = round(grade_12.groupby(["school_name"]).mean(),2)
reading_12 = grade_12_group["reading_score"]

In [18]:
#Pass values in above cell into a dictionary
rscores_by_grade_dict = {"9th":reading_9,"10th":reading_10,"11th":reading_11,"12th":reading_12}

#use dictionary above to create data frame
rscores_by_grade_df = pd.DataFrame(rscores_by_grade_dict)

#remove index name
rscores_by_grade_df.rename_axis(None, inplace=True)

#format scores into floats with 2 decimal places
rscores_by_grade_df.style.format({"9th":"{:,.2f}","10th":"{:,.2f}","11th":"{:,.2f}","12th":"{:,.2f}"})

,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [19]:
school_summary_df.head()

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Bailey High School,District,4976,3124928,628.0,77.05,81.033963,66.68,81.93,54.64
Cabrera High School,Charter,1858,1081356,582.0,83.06,83.975780,94.13,97.04,91.33
Figueroa High School,District,2949,1884411,639.0,76.71,81.158020,65.99,80.74,53.20
Ford High School,District,2739,1763916,644.0,77.10,80.746258,68.31,79.30,54.29
Griffin High School,Charter,1468,917500,625.0,83.35,83.816757,93.39,97.14,90.60


In [20]:
# Create the bins in which Data will be held
# Bins are 0, 59.9, 69.9, 79.9, 89.9, 100.   
bins = [0, 584.99, 629.99, 644.99, 675]

# Create the names for the five bins
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]

#Add column to summary to categorize by spending bin amounts
school_summary_df["Spending Ranges (Per Student)"] = pd.cut(school_summary_df["Per Student Budget"], bins, labels=group_names, include_lowest=True)

#Data to group by bins
math_avg_byspend = school_summary_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
read_avg_byspend = school_summary_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
passing_math_byspend = school_summary_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
passing_reading_byspend = school_summary_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
passing_overall_byspend = school_summary_df.groupby(["Spending Ranges (Per Student)"])["% Passing Overall"].mean()


In [21]:
#create dictionary to hold the values in the above data points
spending_ranges_dict = {"Average Math Score": math_avg_byspend, "Average Reading Score": read_avg_byspend, 
                        "% Passing Math": passing_math_byspend, "% Passing Reading": passing_reading_byspend,
                       "% Passing Overall": passing_overall_byspend}

spending_ranges_df = pd.DataFrame(spending_ranges_dict)

spending_ranges_df.style.format({ "Average Math Score": "{:.2f}","Average Reading Score":"{:.2f}",
                                "% Passing Math":"%{:,.2f}","% Passing Reading":"%{:,.2f}",
                                "% Passing Overall":"%{:,.2f}"})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Spending Ranges (Per Student),,,,,
<$584,83.45,83.93,%93.46,%96.61,%90.37
$585-629,81.90,83.16,%87.13,%92.72,%81.42
$630-644,78.52,81.62,%73.48,%84.39,%62.86
$645-675,77.00,81.03,%66.16,%81.13,%53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [22]:
# Create the bins in which Data will be held
# Bins are 0, 59.9, 69.9, 79.9, 89.9, 100.   
bins = [0, 999, 1999, 5000]

# Create the names for the five bins
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

#Add column to summary to categorize by spending bin amounts
school_summary_df["School Size"] = pd.cut(school_summary_df["Total Students"], bins, labels=group_names, include_lowest=True)

#Data to group by bins
math_avg_byschool = school_summary_df.groupby(["School Size"])["Average Math Score"].mean()
read_avg_byschool = school_summary_df.groupby(["School Size"])["Average Reading Score"].mean()
passing_math_byschool = school_summary_df.groupby(["School Size"])["% Passing Math"].mean()
passing_reading_byschool = school_summary_df.groupby(["School Size"])["% Passing Reading"].mean()
passing_overall_byschool = school_summary_df.groupby(["School Size"])["% Passing Overall"].mean()


In [23]:
#create dictionary to hold the values in the above data points
school_sizes_dict = {"Average Math Score": math_avg_byschool, "Average Reading Score": read_avg_byschool, 
                        "% Passing Math": passing_math_byschool, "% Passing Reading": passing_reading_byschool,
                       "% Passing Overall": passing_overall_byschool}

school_sizes_df = pd.DataFrame(school_sizes_dict)

school_sizes_df.style.format({ "Average Math Score": "{:.2f}","Average Reading Score":"{:.2f}",
                                "% Passing Math":"%{:,.2f}","% Passing Reading":"%{:,.2f}",
                                "% Passing Overall":"%{:,.2f}"})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Size,,,,,
Small (<1000),83.82,83.93,%93.55,%96.10,%89.89
Medium (1000-2000),83.37,83.86,%93.60,%96.79,%90.62
Large (2000-5000),77.74,81.34,%69.96,%82.77,%58.29


## Scores by School Type

* Perform the same operations as above, based on school type

In [24]:
#Group by school type and each average
math_avg_bytype = school_summary_df.groupby(["Type"])["Average Math Score"].mean()
read_avg_bytype = school_summary_df.groupby(["Type"])["Average Reading Score"].mean()
passing_math_bytype = school_summary_df.groupby(["Type"])["% Passing Math"].mean()
passing_reading_bytype = school_summary_df.groupby(["Type"])["% Passing Reading"].mean()
passing_overall_bytype = school_summary_df.groupby(["Type"])["% Passing Overall"].mean()


In [25]:
#create dictionary to hold the values in the above data points
school_type_dict = {"Average Math Score": math_avg_bytype, "Average Reading Score": read_avg_bytype, 
                        "% Passing Math": passing_math_bytype, "% Passing Reading": passing_reading_bytype,
                       "% Passing Overall": passing_overall_bytype}

school_type_df = pd.DataFrame(school_type_dict)
school_type_df.rename_axis("School Type", inplace=True)
school_type_df.style.format({ "Average Math Score": "{:.2f}","Average Reading Score":"{:.2f}",
                                "% Passing Math":"%{:,.2f}","% Passing Reading":"%{:,.2f}",
                                "% Passing Overall":"%{:,.2f}"})


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Type,,,,,
Charter,83.47,83.90,%93.62,%96.59,%90.43
District,76.96,80.97,%66.55,%80.80,%53.67
